In [49]:
# Load Table into pandas dataframe

import pandas as pd
import numpy as np
from pathlib import Path

excel_file = Path.cwd().joinpath(r"C:\Users\joel_\Documents\Python Coding\SQLiteExample\SAP AFO Demo.xlsm")

df = pd.read_excel(excel_file, sheet_name="Sheet1")
df['MonthDate'] = pd.to_datetime(df['Year Period'].astype(str), format='%Y%m')
df

,Calendar Year,Calendar Month Year,Year Period,Company,Country,Business Unit,Group Key Account,Customer Name,Product Code,Product Name,Currency,Product Quantity,Sales Revenue,Cost of Goods Sold,MonthDate
0,2022,12/2022,202212,AU00,Australia,Beverage,Eats International,Dine Divine,BEV005,Fruit Juice,AUD,80,6181.85,2602.23,2022-12-01
1,2022,12/2022,202212,AU00,Australia,Beverage,Eats International,Tasty Trails,BEV002,Lemonade,AUD,82,56435.15,22636.63,2022-12-01
2,2022,12/2022,202212,AU00,Australia,Beverage,Eats International,Tasty Trails,BEV004,Energy Drink,AUD,62,41934.71,17906.86,2022-12-01
3,2022,12/2022,202212,AU00,Australia,Beverage,Gastro Globe,Delightful Dishes,BEV002,Lemonade,AUD,29,18883.47,9865.13,2022-12-01
4,2022,12/2022,202212,AU00,Australia,Beverage,Gastro Globe,Epicurean Eats,BEV002,Lemonade,AUD,9,5033.00,2255.61,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,2022,12/2022,202212,US00,USA,Snacks,Gastro Globe,Culinary Canvas,SNA002,Pretzels,USD,40,13630.85,8326.89,2022-12-01
204,2022,12/2022,202212,US00,USA,Snacks,Global Foods,Bites Bliss,SNA004,Mixed Nuts,USD,55,6282.52,3666.84,2022-12-01
205,2022,12/2022,202212,US00,USA,Snacks,Global Foods,Whisk Wonderland,SNA003,Popcorn,USD,42,9799.33,6391.57,2022-12-01
206,2022,12/2022,202212,US00,USA,Snacks,Snack Planet,Aroma Arcadia,SNA001,Potato Chips,USD,100,11777.19,7058.96,2022-12-01


In [50]:
# Connect to SQL Database
import sqlite3
dbpath = r"C:\Users\joel_\Documents\Python Coding\SQLiteExample\FinanceDB.db"
conn = sqlite3.connect(dbpath)
cur = conn.cursor()
table_name = "SalesTable"

In [51]:
# Define conditions to remove rows from DB
YearPeriod = df['Year Period'].astype(str).unique()
companyList = df['Company'].astype(str).unique()

# Add all conditions into the parameter list
params = np.append(YearPeriod, companyList)

# Build placeholders for the IN clause
placeholder_1 = ', '.join('?' for _ in YearPeriod)
placeholder_2 = ', '.join('?' for _ in companyList)

# Construct SQL Statement to Delete rows fulfilling the conditions
delete_query = f"DELETE FROM {table_name} WHERE [Year Period] IN ({placeholder_1}) AND [Company] IN ({placeholder_2})"

# Check if the table exist
query = f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'"
cur.execute(query)
conn.commit()
result = cur.fetchone()

# Execute DELETE statement if exist
if result is not None:
    cur.execute(delete_query, params)
    conn.commit()

In [52]:
# Insert dataframe/table into SQL Database
df.to_sql(table_name, conn, if_exists='append', index=False)
conn.close()